<a href = "https://www.pieriantraining.com"><img src="../PT Centered Purple.png"> </a>

<em style="text-align:center">Copyrighted by Pierian Training</em>

# Interacting with PostgreSQL Databases

## Azure Actions Covered

* Connecting to a PostgreSQL server
* Reading data from a PostgreSQL database
* Creating tables and schemas in a PostgreSQL database
* Inserting data to a PostgreSQL database
* Updating data in a PostgreSQL database
* Deleting data from a PostgreSQL database

In this lecture, we'll learn how to interact with PostgreSQL servers and databases on Azure with Python.

For this lecture, we don't need any Azure libraries, just `pscyopg2`.

In [27]:
import datetime

import psycopg2

## Connecting to a Server and Reading Data

As we saw in the previous lecture, we can use `psycopg2` to connect to our server. We need the following parameters:

* `host` - Host server. This will be of the form `<server-name>.postgres.database.azure.come`
* `database` - Name of database to connect to
* `user` - User name, we can use our admin user for now
* `password` - Password for the user, we can use our admin user's password

In [3]:
connection = psycopg2.connect(
    host='bens-server.postgres.database.azure.com',
    database='bens-db',
    user='benadmin',
    password='testpassword123!'
)

In [4]:
cursor = connection.cursor()

Let's execute a simple query on our database and return all the data with `cursor.fetchall()`

In [5]:
cursor.execute("""
    select distinct table_schema
    from information_schema.tables;
""")
rows = cursor.fetchall()

In [6]:
rows

[('information_schema',), ('pg_catalog',)]

## Creating Schemas and Tables

To create new schemas and tables, we can write standard PostgreSQL inside of our cursor. Let's create a healthcare-related table in a new schema.

In [25]:
cursor.execute("""
    create schema if not exists hospital;
    create table hospital.patients (
        id serial primary key,
        name varchar(50),
        date_of_birth date
    );
""")

Let's see our new list of schemas using the query we used above.

In [26]:
cursor.execute("""
    select distinct table_schema
    from information_schema.tables;
""")
rows = cursor.fetchall()
rows

[('public',), ('pg_catalog',), ('information_schema',), ('hospital',)]

## Inserting Data

Again, to insert data, we can just execute an `INSERT` statement inside of our cursor. **Note:** We can reference variables inside our query and then pass the values as parameters to protect against SQL injection.

In [28]:
cursor.execute("""
    insert into hospital.patients (name, date_of_birth) values
        (%(name)s, %(dob)s)
""", {'name': 'John Doe', 'dob': datetime.date(1999, 1, 1)})

Let's get our new data from the `patients` table.

In [29]:
cursor.execute("""
    select *
    from hospital.patients;
""")
rows = cursor.fetchall()
rows

[(1, 'John Doe', datetime.date(1999, 1, 1))]

## Updating Data

Updating data works much the same as inserting data. Let's update the patient data we inserted into our new table.

In [30]:
cursor.execute("""
    update hospital.patients
        set date_of_birth = %(dob)s
        where id = %(id)s;
""", {'dob': datetime.date(1998, 12, 31), 'id': 1})

We can check that the changes took place in the DB.

In [31]:
cursor.execute("""
    select *
    from hospital.patients;
""")
cursor.fetchone()

(1, 'John Doe', datetime.date(1998, 12, 31))

## Deleting Data

Deleting data works just like updating and inserting data, with variables/parameters to avoid SQL injection.

In [32]:
cursor.execute("""
    delete from hospital.patients
        where id = %(id)s
""", {'id': 1})

Now there are no more patients in our `patients` table.

In [33]:
cursor.execute("""
    select *
    from hospital.patients;
""")
cursor.fetchone()

Finally, let's close the cursor and connection.

In [34]:
cursor.close()
connection.close()